In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
from IPython.display import clear_output
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [2]:
start_data_season = {
    'M': 2003,
    'W': 2009
}
classifiers = {
    'M': RandomForestClassifier(max_depth=80, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1600, random_state=0),
    'W': RandomForestClassifier(max_depth=100, max_features=None, min_samples_leaf=4,
                       min_samples_split=5, n_estimators=2000, random_state=0)
}
columns = [
            'Season', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 
            'ASeedNum', 'ALastTournPct', 'ARegSznPct',
            'APts', 'APtsOpp', 'AFG', 'AFG3', 'AFT', 'AAst', 'ATO', 'AOR', 'ADR', 'AStl', 'ABlk', 'APF',
            'BSeedNum', 'BLastTournPct', 'BRegSznPct',
            'BPts', 'BPtsOpp', 'BFG', 'BFG3', 'BFT', 'BAst', 'BTO', 'BOR', 'BDR', 'BStl', 'BBlk', 'BPF',
            'Winner'
        ]
season = 2025

In [3]:
def get_teams_info(gender):
    teams_file = f'/kaggle/input/march-machine-learning-mania-2025/{gender}Teams.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    return teams_df

m_teams_info_df = get_teams_info('M')
w_teams_info_df = get_teams_info('W')

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025
...,...,...,...,...
375,1476,Stonehill,2023,2025
376,1477,East Texas A&M,2023,2025
377,1478,Le Moyne,2024,2025
378,1479,Mercyhurst,2025,2025


,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M
...,...,...
373,3476,Stonehill
374,3477,East Texas A&M
375,3478,Le Moyne
376,3479,Mercyhurst


In [4]:
def get_seed_number(seed):
    if 'a' in seed or 'b' in seed:
        return 17
    return int(seed[1:])

def generate_numeric_cols(df, team):
    df[f'{team}FG'] = (df[f'{team}FGM'] * 100) / df[f'{team}FGA']
    df[f'{team}FG3'] = (df[f'{team}FGM3'] * 100) / df[f'{team}FGA3']
    df[f'{team}FT'] = (df[f'{team}FTM'] * 100) / df[f'{team}FTA']

def get_files_by_competition(gender):
    # Getting teams df
    teams_file = f'/kaggle/input/march-machine-learning-mania-2025/{gender}TeamConferences.csv'
    teams_df = pd.read_csv(teams_file)
    display(teams_df)
    
    # Getting regular season games df
    reg_szn_file = f'/kaggle/input/march-machine-learning-mania-2025/{gender}RegularSeasonDetailedResults.csv'
    reg_szn_df = pd.read_csv(reg_szn_file)
    generate_numeric_cols(reg_szn_df, 'W')
    generate_numeric_cols(reg_szn_df, 'L')
    display(reg_szn_df)
    
    # Getting tournament season games df
    tourn_file = f'/kaggle/input/march-machine-learning-mania-2025/{gender}NCAATourneyCompactResults.csv'
    tourn_df = pd.read_csv(tourn_file)
    display(tourn_df)
    
    # Getting seeding df
    seed_file = f'/kaggle/input/march-machine-learning-mania-2025/{gender}NCAATourneySeeds.csv'
    seed_df = pd.read_csv(seed_file)
    seed_df['SeedNum'] = seed_df.Seed.apply(lambda x: get_seed_number(x))
    display(seed_df)
    
    return teams_df, reg_szn_df, tourn_df, seed_df

In [5]:
def get_reg_szn_stats(season, team, reg_szn_df):
    w_games = reg_szn_df[reg_szn_df['WTeamID'] == team].rename(columns = {
        'WFG': 'FG', 'WFG3': 'FG3', 'WFT': 'FT', 'WAst': 'Ast', 'WTO': 'TO', 'WOR': 'OR', 'WDR': 'DR', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
        'WScore': 'Score',
        
        'LFG': 'OppFG', 'LFG3': 'OppFG3', 'LFT': 'OppFT', 'LAst': 'OppAst', 'LTO': 'OppTO', 'LOR': 'OppOR', 'LDR': 'OppDR', 'LStl': 'OppStl', 'LBlk': 'OppBlk', 'LPF': 'OppPF',
        'LScore': 'OppScore',
        })
    w_games['Won'] = 1
    
    l_games = reg_szn_df[reg_szn_df['LTeamID'] == team].rename(columns = {
        'LFG': 'FG', 'LFG3': 'FG3', 'LFT': 'FT', 'LAst': 'Ast', 'LTO': 'TO', 'LOR': 'OR', 'LDR': 'DR', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
        'LScore': 'Score',
        
        'WFG': 'OppFG', 'WFG3': 'OppFG3', 'WFT': 'OppFT', 'WAst': 'OppAst', 'WTO': 'OppTO', 'WOR': 'OppOR', 'WDR': 'OppDR', 'WStl': 'OppStl', 'WBlk': 'OppBlk', 'WPF': 'OppPF',
        'WScore': 'OppScore',
        })
    l_games['Won'] = 0
    
    games = pd.concat([w_games, l_games], axis=0, ignore_index=True)
    
    reg_szn_pct = (len(w_games) * 100) / len(games)
    
    return [   
               reg_szn_pct, 
               games['Score'].mean(), games['OppScore'].mean(), games['FG'].mean(), games['FG3'].mean(), games['FT'].mean(), games['Ast'].mean(), games['TO'].mean(), 
               games['OR'].mean(), games['DR'].mean(), games['Stl'].mean(), games['Blk'].mean(), games['PF'].mean()
           ]

In [6]:
def get_last_tourn_pct(season, team, tourn_df):
    wins = len(tourn_df[(tourn_df['Season'] == season-1) & (tourn_df['WTeamID'] == team)])
    total_games = len(tourn_df[(tourn_df['Season'] == season-1) & ((tourn_df['WTeamID'] == team) | (tourn_df['LTeamID'] == team))])
    
    if not total_games: return 0
    return (wins * 100) / total_games

In [7]:
def set_team_name(team_id, teams_df):
    return teams_df[teams_df['TeamID'] == team_id].reset_index().loc[0, 'TeamName']

def get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df):
    a_reg_szn_stats = get_reg_szn_stats(season, team_a, reg_szn_df)
    b_reg_szn_stats = get_reg_szn_stats(season, team_b, reg_szn_df)

    try:
        a_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_a)].reset_index().loc[0, 'SeedNum']
    except:
        a_seed = 24
    try:
        b_seed = seed_df[(seed_df['Season'] == season) & (seed_df['TeamID'] == team_b)].reset_index().loc[0, 'SeedNum']
    except:
        b_seed = 24

    a_last_tourn_pct = get_last_tourn_pct(season, team_a, tourn_df)
    b_last_tourn_pct = get_last_tourn_pct(season, team_b, tourn_df)

    stats_a = [a_seed, a_last_tourn_pct] + (a_reg_szn_stats)
    stats_b = [b_seed, b_last_tourn_pct] + (b_reg_szn_stats)
    
    teams_info_df = m_teams_info_df if gender == 'M' else w_teams_info_df
    team_a_name = set_team_name(team_a, teams_info_df)
    team_b_name = set_team_name(team_b, teams_info_df)
    
    return [season, team_a, team_b, team_a_name, team_b_name] + stats_a + stats_b

def build_dataset(gender, start_data_season, reg_szn_df, tourn_df, seed_df, end_season = 2025):
    data = []

    for season in range(start_data_season + 1, end_season + 1):
        tourney_games = tourn_df[tourn_df['Season'] == season].reset_index(drop=True)
        for idx, g in tourney_games.iterrows():

            clear_output(wait=True)
            print(f"{season}: {idx}/{len(tourney_games)}")

            team_a = min([g['WTeamID'], g['LTeamID']])
            team_b = max([g['WTeamID'], g['LTeamID']])

            if team_a == g['WTeamID']:
                winner = 'A'
                team_a_score = g['WScore']
                team_b_score = g['LScore']
            else:
                winner = 'B'
                team_a_score = g['LScore']
                team_b_score = g['WScore']

            print(f"{team_a} x {team_b}")

            game_stats = get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df)

            data.append(game_stats + [winner])

    data_df = pd.DataFrame(data, columns=columns)
    display(data_df)
    
    return data_df

In [8]:
def train_model(x_train, y_train, gender):
    classifier = classifiers[gender]
    classifier.fit(x_train, y_train)
    return classifier

def predict_seasons(gender, start_data_season, data_df, detailed_results):

    acc_sum = 0
    seasons_count = 0

    for season in range(start_data_season + 2, 2026):
        data_train = data_df[(data_df['Season'] < season)].reset_index(drop=True)
        data_test = data_df[(data_df['Season'] == season)].reset_index(drop=True)

        if not len(data_test):
            continue

        x_train = data_train.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_train = data_train.loc[:, 'Winner']
            
        classifier = train_model(x_train, y_train, gender)

        x_test = data_test.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
        y_test = data_test.loc[:, 'Winner']

        predictions = classifier.predict(x_test)
        
        acc = accuracy_score(y_test, predictions)
        acc_sum += acc
        seasons_count += 1
        
        if detailed_results:
            print(f'\nResults for {gender} season {season}:')
            print('Accuracy predictions:', acc)

            cm = confusion_matrix(y_test, predictions)
            print('Confusion matrix:')
            cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classifier.classes_)
            cm_disp.plot()
            plt.show()

    print(f'\n\n{gender} Mean Accuracy: {acc_sum/seasons_count}')
    
    return classifier

In [9]:
def get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df):
    data = []
    teams = seed_df[seed_df['Season'] == 2024].TeamID.unique() # Change for 2025 later when seeding is available
    teams.sort()
    
    for idx, team_a in enumerate(teams):
        for idx_b in range(idx + 1, len(teams)):
            team_b = teams[idx_b]
            
            clear_output(wait=True)
            print(f"{idx}/{len(teams)}")
            print(f"{team_a} x {team_b}")
            
            game_stats = get_game_stats(gender, season, team_a, team_b, reg_szn_df, seed_df, tourn_df)
            data.append(game_stats)
            
    data_df = pd.DataFrame(data, columns=columns[:-1])
    data_to_predict = data_df.drop(['TeamNameA', 'TeamNameB'], axis=1)
    
    probs = classifier.predict_proba(data_to_predict)
    pred = classifier.predict(data_to_predict)

    data_df['AProb'] = probs[:, 0]
    data_df['BProb'] = probs[:, 1]
    data_df['Pred'] = pred
    
    return data_df

In [10]:
def pipeline(gender, detailed_results=False):
    teams_df, reg_szn_df, tourn_df, seed_df = get_files_by_competition(gender)
    data_df = build_dataset(gender, start_data_season[gender], reg_szn_df, tourn_df, seed_df)
    classifier = predict_seasons(gender, start_data_season[gender], data_df, detailed_results)
    current_szn_df = get_current_predictions(gender, classifier, reg_szn_df, tourn_df, seed_df)
    display(current_szn_df)
    return current_szn_df, teams_df

In [11]:
m_current_szn_df, m_teams_df = pipeline('M')

66/68
1458 x 1463


,Season,TeamA,TeamB,TeamNameA,TeamNameB,ASeedNum,ALastTournPct,ARegSznPct,APts,APtsOpp,...,BAst,BTO,BOR,BDR,BStl,BBlk,BPF,AProb,BProb,Pred
0,2025,1103,1104,Akron,Alabama,24,0.0,65.726375,72.348378,67.359661,...,12.925000,13.316667,11.487500,25.220833,6.705556,4.500000,18.012500,0.436217,0.563783,B
1,2025,1103,1112,Akron,Arizona,24,0.0,65.726375,72.348378,67.359661,...,15.569293,12.652174,11.187500,25.532609,6.498641,3.721467,17.038043,0.261372,0.738628,B
2,2025,1103,1120,Akron,Auburn,24,0.0,65.726375,72.348378,67.359661,...,13.727910,13.328191,11.412342,23.563815,7.527349,4.483871,19.273492,0.466702,0.533298,B
3,2025,1103,1124,Akron,Baylor,24,0.0,65.726375,72.348378,67.359661,...,14.046579,13.090247,11.933042,23.257642,6.973799,3.976710,17.754003,0.306702,0.693298,B
4,2025,1103,1129,Akron,Boise St,24,0.0,65.726375,72.348378,67.359661,...,13.121212,12.603175,9.878788,24.232323,6.082251,2.568543,17.930736,0.474266,0.525734,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2025,1447,1458,Wagner,Wisconsin,24,50.0,51.745068,67.757208,68.324734,...,12.749658,9.722298,9.761970,23.704514,5.554036,3.131327,15.452804,0.277211,0.722789,B
2274,2025,1447,1463,Wagner,Yale,24,50.0,51.745068,67.757208,68.324734,...,13.832237,13.569079,10.001645,24.797697,6.226974,3.514803,17.868421,0.334985,0.665015,B
2275,2025,1450,1458,Washington St,Wisconsin,24,50.0,48.648649,68.186344,67.832148,...,12.749658,9.722298,9.761970,23.704514,5.554036,3.131327,15.452804,0.348856,0.651144,B
2276,2025,1450,1463,Washington St,Yale,24,50.0,48.648649,68.186344,67.832148,...,13.832237,13.569079,10.001645,24.797697,6.226974,3.514803,17.868421,0.441369,0.558631,B


In [12]:
w_current_szn_df, w_teams_df = pipeline('W')

66/68
3453 x 3465


,Season,TeamA,TeamB,TeamNameA,TeamNameB,ASeedNum,ALastTournPct,ARegSznPct,APts,APtsOpp,...,BAst,BTO,BOR,BDR,BStl,BBlk,BPF,AProb,BProb,Pred
0,2025,3104,3112,Alabama,Arizona,24,50.0,54.037267,67.472050,64.236025,...,12.734864,15.206681,10.668058,23.916493,8.620042,3.334029,16.780793,0.494944,0.505056,B
1,2025,3104,3120,Alabama,Auburn,24,50.0,54.037267,67.472050,64.236025,...,12.783439,16.197452,13.053079,21.817410,10.201699,3.690021,17.749469,0.540368,0.459632,A
2,2025,3104,3124,Alabama,Baylor,24,50.0,54.037267,67.472050,64.236025,...,19.512720,14.082192,13.792564,29.808219,8.043053,5.763209,15.336595,0.307676,0.692324,B
3,2025,3104,3151,Alabama,Chattanooga,24,50.0,54.037267,67.472050,64.236025,...,12.336820,14.016736,9.903766,24.941423,6.734310,3.610879,13.903766,0.360821,0.639179,B
4,2025,3104,3160,Alabama,Colorado,24,50.0,54.037267,67.472050,64.236025,...,13.697674,15.784355,12.469345,24.875264,8.395349,3.600423,18.215645,0.375810,0.624190,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2025,3439,3453,Virginia Tech,WI Green Bay,24,50.0,55.741127,66.066806,61.858038,...,16.528926,13.198347,10.611570,24.365702,8.923554,2.407025,15.925620,0.552606,0.447394,A
2274,2025,3439,3465,Virginia Tech,Cal Baptist,24,50.0,55.741127,66.066806,61.858038,...,18.111675,14.893401,8.390863,28.624365,8.365482,3.360406,17.827411,0.488456,0.511544,B
2275,2025,3452,3453,West Virginia,WI Green Bay,24,50.0,69.556452,69.094758,58.659274,...,16.528926,13.198347,10.611570,24.365702,8.923554,2.407025,15.925620,0.544977,0.455023,A
2276,2025,3452,3465,West Virginia,Cal Baptist,24,50.0,69.556452,69.094758,58.659274,...,18.111675,14.893401,8.390863,28.624365,8.365482,3.360406,17.827411,0.452744,0.547256,B


In [13]:
def set_game_id(row):
    return f"{row['Season']}_{row['TeamA']}_{row['TeamB']}"

def set_impossible_games_probs(gender, current_szn_df, teams_df):
    teams_a = current_szn_df['TeamA'].unique().tolist()
    teams_b = current_szn_df['TeamB'].unique().tolist()
    tourney_teams = set(teams_a + teams_b)

    teams = teams_df[teams_df['Season'] == season]['TeamID'].unique().tolist()

    data = []

    for idx, team_a in enumerate(teams):
        for idx_b in range(idx + 1, len(teams)):
            team_b = teams[idx_b]

            if team_a in tourney_teams and team_b in tourney_teams:
                continue
                
            teams_info_df = m_teams_info_df if gender == 'M' else w_teams_info_df
            # There is no reason for setting team names or probabilities if these matchups will never happen in the tournament
            team_a_name = None # set_team_name(team_a, teams_info_df)
            team_b_name = None # set_team_name(team_b, teams_info_df)

            game_id = f"{season}_{team_a}_{team_b}"
            data.append([game_id, team_a, team_b, team_a_name, team_b_name, 0.5])

    data_df = pd.DataFrame(data, columns=['ID', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 'Pred'])
    return data_df

current_szn_df = pd.concat([m_current_szn_df, w_current_szn_df], axis=0).reset_index(drop=True)
current_szn_df['ID'] = current_szn_df.apply(lambda row: set_game_id(row), axis=1)

m_impossible_df = set_impossible_games_probs('M', m_current_szn_df, m_teams_df)
w_impossible_df = set_impossible_games_probs('W', w_current_szn_df, w_teams_df)

final_df = current_szn_df.loc[:, ['ID', 'TeamA', 'TeamB', 'TeamNameA', 'TeamNameB', 'AProb']].rename({'AProb': 'Pred'}, axis=1)
final_df = pd.concat([final_df, m_impossible_df, w_impossible_df], axis=0).sort_values(by='ID').reset_index(drop=True)

display(final_df)

,ID,TeamA,TeamB,TeamNameA,TeamNameB,Pred
0,2025_1101_1102,1101,1102,None,None,0.5
1,2025_1101_1103,1101,1103,None,None,0.5
2,2025_1101_1104,1101,1104,None,None,0.5
3,2025_1101_1105,1101,1105,None,None,0.5
4,2025_1101_1106,1101,1106,None,None,0.5
...,...,...,...,...,...,...
131402,2025_3477_3479,3477,3479,None,None,0.5
131403,2025_3477_3480,3477,3480,None,None,0.5
131404,2025_3478_3479,3478,3479,None,None,0.5
131405,2025_3478_3480,3478,3480,None,None,0.5


In [14]:
team_a = 'Baylor'
team_b = 'Duke'

# seed_file = f'/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv'
# seed_df = pd.read_csv(seed_file)
# seed_df['SeedNum'] = seed_df.Seed.apply(lambda x: get_seed_number(x)) # 1297 1224
# display(seed_df[(seed_df['SeedNum'] == 17) & (seed_df['Season'] == season)])

game = final_df[((final_df['TeamNameA'].str.contains(team_a)) & (final_df['TeamNameB'].str.contains(team_b))) |
               ((final_df['TeamNameB'].str.contains(team_a)) & (final_df['TeamNameA'].str.contains(team_b)))]

display(game)

,ID,TeamA,TeamB,TeamNameA,TeamNameB,Pred
7123,2025_1124_1181,1124,1181,Baylor,Duke,0.426298
73148,2025_3124_3181,3124,3181,Baylor,Duke,0.410855


In [15]:
def create_submission_file(df):
    df_filtered = df.loc[:, ['ID', 'Pred']]
    df_filtered.to_csv('submission.csv', index=False)
    
create_submission_file(final_df)

In [16]:
def get_rf_best_parameters(gender):
    teams_df, reg_szn_df, tourn_df, seed_df = get_files_by_competition(gender)

    data_df = build_dataset(gender, start_data_season[gender], reg_szn_df, tourn_df, seed_df, end_season = 2022) 
    x = data_df.drop(['TeamNameA', 'TeamNameB', 'Winner'], axis=1)
    y = data_df.loc[:, 'Winner']

    from sklearn.model_selection import RandomizedSearchCV
    rs_optimizer = RandomForestClassifier()

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['sqrt', 'log2', None]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                    'max_features': max_features,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'bootstrap': bootstrap}

    rf_random = RandomizedSearchCV(estimator = rs_optimizer, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)

    rf_random.fit(x, y)

    best_random = rf_random.best_estimator_
    best_parameters = rf_random.cv_results_
    #     print(best_parameters)
    print(best_random)
    
# get_rf_best_parameters('M')